In [ ]:
1. spillover
2. reservoir
3. more than 1 harvester per parcel
4. maker installation

In [1]:
import pandas as pd

In [22]:
!export PATH=/Library/TeX/texbin:$PATH


### reading data and preprocessing it

In [2]:
avg_parcel_yield = pd.read_csv("data/avg_parcel_yield.csv")
alchemica_supply = pd.read_csv("data/alchemica_supply.csv", skipinitialspace=True)

In [3]:
avg_parcel_yield = avg_parcel_yield.replace(',','', regex=True)
alchemica_supply = alchemica_supply.replace(',','', regex=True)

In [4]:
fud_harvester = pd.read_csv("data/fud_harvester.csv")
fomo_harvester = pd.read_csv("data/fomo_harvester.csv")
alpha_harvester = pd.read_csv("data/alpha_harvester.csv")
kek_harvester = pd.read_csv("data/kek_harvester.csv")

In [5]:
fud_harvester = fud_harvester.replace(',','', regex=True).astype(int)
fomo_harvester = fomo_harvester.replace(',','', regex=True).astype(int)
alpha_harvester = alpha_harvester.replace(',','', regex=True).astype(int)
kek_harvester = kek_harvester.replace(',','', regex=True).astype(int)

### exchange rates among different alchemica

the exchanges rates will be determined by some oracle in future which will take data from secondary markets <br>
here we all using total supply as a proxy for exchange rates

In [6]:
alchemica_supply.head()

,Type,FUD,FOMO,ALPHA,KEK
0,Total Supply,100000000000,50000000000,25000000000,10000000000
1,Channeling (25%),25000000000,12500000000,6250000000,2500000000
2,Ecosystem (10%),10000000000,5000000000,2500000000,1000000000
3,Gameplay - Boosts (5%),5000000000,2500000000,1250000000,500000000
4,Gameplay -Other (10%),10000000000,5000000000,2500000000,1000000000


In [7]:
fud_factor = int(alchemica_supply["FUD"][0])/int(alchemica_supply['FUD'][0])
fomo_factor = int(alchemica_supply["FUD"][0])/int(alchemica_supply['FOMO'][0])
alpha_factor = int(alchemica_supply["FUD"][0])/int(alchemica_supply['ALPHA'][0])
kek_factor = int(alchemica_supply["FUD"][0])/int(alchemica_supply['KEK'][0])

print("FUD to FUD factor --> {}".format(fud_factor))
print("FOMO to FUD factor --> {}".format(fomo_factor))
print("ALPHA to FUD factor --> {}".format(alpha_factor))
print("KEK to FUD factor --> {}".format(kek_factor))

FUD to FUD factor --> 1.0
FOMO to FUD factor --> 2.0
ALPHA to FUD factor --> 4.0
KEK to FUD factor --> 10.0


### creating data for different parcel types

In [8]:
avg_parcel_yield.head()

,Parcel Type,FUD,FOMO,ALPHA,KEK
0,Humble (Small),7077,3539,1769,708
1,Reasonable (Medium),28308,14154,7077,2831
2,Spacious (Large),226465,113233,56616,22647


In [9]:
fud_yield = {
    "Humble (Small)": 7077,
    "Reasonable (Medium)": 28308,
    "Spacious (Large)": 226465
}

fomo_yield = {
    "Humble (Small)": 3539,
    "Reasonable (Medium)": 14154,
    "Spacious (Large)": 113233
}

alpha_yield = {
    "Humble (Small)": 1769,
    "Reasonable (Medium)": 7077,
    "Spacious (Large)": 56616
}

kek_yield = {
    "Humble (Small)": 708,
    "Reasonable (Medium)": 2831,
    "Spacious (Large)": 22647
}

### helper functions

In [10]:
def equilibrium_level_for_installation(installation, given_yield):
    df = installation.copy()
    df["total_build_cost"] =  df.apply(lambda row: fud_factor*row["FUD"]+fomo_factor*row['FOMO']+alpha_factor*row['ALPHA']+kek_factor*row['KEK'], axis=1)
    temp_df = df.loc[df['total_build_cost'] < given_yield]
    max_level = max(temp_df['Level'])
    return max_level
    

### Results

results for fud harvestor

In [11]:
for key, value in fud_yield.items():
    max_level = equilibrium_level_for_installation(fud_harvester, value*fud_factor)
    print("level for {} --> {}".format(key, max_level))

level for Humble (Small) --> 5
level for Reasonable (Medium) --> 7
level for Spacious (Large) --> 9


results for fomo harvestor

In [12]:
for key, value in fomo_yield.items():
    max_level = equilibrium_level_for_installation(fomo_harvester, value*fomo_factor)
    print("level for {} --> {}".format(key, max_level))

level for Humble (Small) --> 5
level for Reasonable (Medium) --> 7
level for Spacious (Large) --> 9


results for alpha harvestor

In [13]:
for key, value in alpha_yield.items():
    max_level = equilibrium_level_for_installation(alpha_harvester, value*alpha_factor)
    print("level for {} --> {}".format(key, max_level))

level for Humble (Small) --> 5
level for Reasonable (Medium) --> 7
level for Spacious (Large) --> 9


results for kek harvestor

In [14]:
for key, value in kek_yield.items():
    max_level = equilibrium_level_for_installation(kek_harvester, value*kek_factor)
    print("level for {} --> {}".format(key, max_level))

level for Humble (Small) --> 5
level for Reasonable (Medium) --> 7
level for Spacious (Large) --> 9


### explanation about how equilibrium_level_for_installation() function works

In [15]:
df = kek_harvester.copy()

calculate the total cost which is required to build the installation. Since there are multiple tokens. We are calcuating the cost in fud, using the token conversion factors we calcualted earlier

In [19]:
df["total_build_cost_in_fud"] =  df.apply(lambda row: fud_factor*row["FUD"]+fomo_factor*row['FOMO']+alpha_factor*row['ALPHA']+kek_factor*row['KEK'], axis=1)
df.head(10)

,Level,FUD,FOMO,ALPHA,KEK,Harvest Rate (KEK) / Day,Built Time (Blocks),Buid Time (Days),total_build_cost_in_fud
0,1,110,70,0,3,1,0,0,280.0
1,2,220,140,0,5,2,8800,0,550.0
2,3,440,275,0,11,5,22000,0,1100.0
3,4,825,700,0,55,9,44000,1,2775.0
4,5,1650,1375,0,110,12,66000,1,5500.0
5,6,3300,2750,0,220,18,88000,2,11000.0
6,7,5500,6900,0,825,25,175000,4,27550.0
7,8,11000,13750,0,1650,36,260000,6,55000.0
8,9,11000,27500,0,4400,49,440000,11,110000.0


In [20]:
avg_parcel_yield.head()

,Parcel Type,FUD,FOMO,ALPHA,KEK
0,Humble (Small),7077,3539,1769,708
1,Reasonable (Medium),28308,14154,7077,2831
2,Spacious (Large),226465,113233,56616,22647


In [21]:
for key, value in kek_yield.items():
    max_level = equilibrium_level_for_installation(fomo_harvester, value*kek_factor)
    print("level for {} --> {}".format(key, max_level))

level for Humble (Small) --> 5
level for Reasonable (Medium) --> 7
level for Spacious (Large) --> 9
